# Problem 4
## add parameter
* let $BackOrderCost_i,\ BackOrderPercent_i$ be the cost of back order and the probability of back order, respectively.

## add decision variable
* let $y_{ij}$ be the stock level of product $i$ of month $j$
* let $w_{ij}$ be the amount of unfulfill demand of product $i$ of month $j$
* let $ywBin_{ij}$ be the helper variable for $y_{ij},\ w_{ij}$ for let them be<br>
$y_{ij} = max\{0,\ InventoryExpr_{ij}\}$<br>
$w_{ij} = max\{0,\ -InventoryExpr_{ij}\}$<br>

## Expr
* let $BackOrderExpr_{ij}$ be the amout of back order of product $i$ of month $j$<br>
$BackOrderExpr_{ij} = w_{ij} BackOrderPercent_i$<br>
* let $InventoryExpr_{ij}$ be the expression of ending inventory of product $i$ of month $j$<br>
$j=0:\ InventoryExpr_{i0} = I_i - D_{i0}$<br>
$j=1:\ InventoryExpr_{i1} = y_{i0} + x_{i00} + Transit_{i1} - D_{i1} - BackOrderExpr_{i0}$<br>
$j=2:\ InventoryExpr_{i2} = y_{i1} + x_{i10} + x_{i01} + Transit_{i2} - D_{i2} - BackOrderExpr_{i1}$<br>
$j \ge 3:\ InventoryExpr_{ij} = y_{i(j-1)} + x_{i(j-1)0} + x_{i(j-2)1} + x_{i(j-3)2} - D_{ij} - BackOrderExpr_{i(j-1)}$<br>

## add to obj function
- back order cost = $\sum_{i=0}^N \sum_{j=0}^M BackOrderExpr_{ij} BackOrderCost_i$
- lost slae cost = $\sum_{i=0}^N \sum_{j=0}^M w_{ij} (1 - BackOrderPercent_i) LostSaleCost_i$

## add constrain
$ 0 \le y_{ij}$<br>
$InventoryExpr_{ij} \le y_{ij}$<br>
$y_{ij} \le 0 + x_{max} \times ywBin_{ij}$<br>
$y_{ij} \le InventoryExpr_{ij} + x_{max} \times (1 - ywBin_{ij})$<br>
$ 0 \le w_{ij}$<br>
$-InventoryExpr_{ij} \le w_{ij}$<br>
$w_{ij} \le 0 + x_{max} \times (1 - ywBin_{ij})$<br>
$w_{ij} \le -InventoryExpr_{ij} + x_{max} \times  ywBin_{ij}$<br>

In [2]:
import gurobipy as gb
from gurobipy import GRB
from gurobipy import quicksum
import pandas as pd
import numpy as np

In [3]:
## setting parameter
N, M, K = 10, 6, 3
D_ij = pd.read_excel('data.xlsx', sheet_name='Demand', index_col='Product').to_numpy()
I_i = pd.read_excel('data.xlsx', sheet_name='Initial inventory', index_col='Product').to_numpy().squeeze()
BuyCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Purchasing cost'].to_numpy().squeeze()
HoldCost_i = pd.read_excel('data.xlsx', sheet_name='Price and cost', index_col='Product')['Holding'].to_numpy().squeeze()
Transit_ij = pd.read_excel('data.xlsx', sheet_name='In-transit', index_col='Product').to_numpy().squeeze()
FixedShipCost_k = (100, 80, 50)
VarShipCost_ik = pd.read_excel('data.xlsx', sheet_name='Shipping cost', index_col='Product').to_numpy().squeeze()
x_max = sum([sum(i) for i in D_ij]) + sum(I_i)
ContainerCapacity = 30
ContainerCost = 2750
CBM_i = pd.read_excel('data.xlsx', sheet_name='Size', index_col='Product').to_numpy().squeeze()
BackOrderCost_i = pd.read_excel('data.xlsx', sheet_name='Shortage', index_col='Product')['Backorder'].to_numpy()
BackOrderPercent_i = pd.read_excel('data.xlsx', sheet_name='Shortage', index_col='Product')['Backorder percentage'].to_numpy()
LostSaleCost_i = pd.read_excel('data.xlsx', sheet_name='Shortage', index_col='Product')['Lost sales'].to_numpy()

In [4]:
m = gb.Model("Problem4Model")

Academic license - for non-commercial use only - expires 2021-05-20
Using license file /Users/eason/gurobi.lic


In [5]:
#add decision variable
x = m.addVars(N, M, K, vtype=GRB.INTEGER, name='x_ijk')
Abin = m.addVars(M, K, vtype=GRB.BINARY, name='z_jk')
ContainerCnt = m.addVars(M, vtype=GRB.INTEGER, name='ContainerCnt_j')
StockLevel = m.addVars(N, M, vtype=GRB.CONTINUOUS, name='StockLevel')
Shortage = m.addVars(N, M, vtype=GRB.CONTINUOUS, name='Shortage')
Bbin = m.addVars(N, M, vtype=GRB.BINARY, name='Bbin')

In [6]:
#add Expr
ValueInOceanExpr_j = [
    quicksum(x[i,j,2]*CBM_i[i] for i in range(N))
for j in range(M)]

BackOrderExpr_ij = [
    [
        Shortage[i,j] * BackOrderPercent_i[i]
    for j in range(M)]
for i in range(N)]

LostSaleCntExpr_ij = [
    [
        Shortage[i,j] *(1 - BackOrderPercent_i[i])
    for j in range(M)]
for i in range(N)]

In [7]:
#add to objctive function
purchaseCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * BuyCost_i[i]
            for k in range(K))
        for j in range(M))
    for i in range(N))
)

FixedCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            Abin[j,k] * FixedShipCost_k[k]
        for k in range(K))
    for j in range(M))
)

VariableCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            quicksum(
                x[i,j,k] * VarShipCost_ik[i][k]
            for k in range(K))
        for i in range(N))
    for j in range(M))
)

InventoryCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            StockLevel[i,j]
        for j in range(M)) * HoldCost_i[i]
    for i in range(N))
)

allContainerCostExpr =  gb.LinExpr(
    quicksum(ContainerCnt[j] * ContainerCost for j in range(M))
)

BackOrderCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            BackOrderExpr_ij[i][j] * BackOrderCost_i[i]
        for j in range(M))
    for i in range(N))
)

LostSaleCostExpr = gb.LinExpr(
    quicksum(
        quicksum(
            LostSaleCntExpr_ij[i][j] * LostSaleCost_i[i]
        for j in range(M))
    for i in range(N))
)

m.setObjective(
    purchaseCostExpr + FixedCostExpr + VariableCostExpr + InventoryCostExpr + allContainerCostExpr + BackOrderCostExpr + LostSaleCostExpr
)

In [8]:
#Constrains
_ = m.addConstrs(
    quicksum(x[i,j,k] for i in range(0, N)) / x_max <= Abin[j,k]
for j in range(0, M)
for k in range(0, K)
)
_ = m.addConstrs(
     ValueInOceanExpr_j[j] / ContainerCapacity <= ContainerCnt[j]
 for j in range(M))

_ = m.addConstrs(StockLevel[i,0] - Shortage[i,0] == I_i[i] - D_ij[i][0] for i in range(N))
_ = m.addConstrs(StockLevel[i,1] - Shortage[i,1] == StockLevel[i,0] + x[i,0,0] + Transit_ij[i][1] - D_ij[i][1] - Shortage[i,0] * BackOrderPercent_i[i] for i in range(N))
_ = m.addConstrs(StockLevel[i,2] - Shortage[i,2] == StockLevel[i,1] + x[i,1,0] + x[i,0,1] + Transit_ij[i][2] - D_ij[i][2] - Shortage[i,1] * BackOrderPercent_i[i] for i in range(N))
_ = m.addConstrs(StockLevel[i,j] - Shortage[i,j] == StockLevel[i,j-1] + x[i,j-1,0] + x[i,j-2,1] + x[i,j-3,2] - D_ij[i][j] - Shortage[i,j-1] * BackOrderPercent_i[i] for i in range(N) for j in range(3,M))

_ = m.addConstrs(StockLevel[i,j] <= x_max * (1-Bbin[i,j]) for i in range(N) for j in range(M))
_ = m.addConstrs(Shortage[i,j] <= x_max * Bbin[i,j] for i in range(N) for j in range(M))

In [9]:
m.optimize()

Gurobi Optimizer version 9.1.1 build v9.1.1rc0 (mac64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 204 rows, 384 columns and 839 nonzeros
Model fingerprint: 0x041277b1
Variable types: 120 continuous, 264 integer (78 binary)
Coefficient statistics:
  Matrix range     [8e-05, 1e+04]
  Objective range  [4e+01, 2e+04]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+04]
Found heuristic solution: objective 1.655854e+07
Presolve removed 110 rows and 170 columns
Presolve time: 0.00s
Presolved: 94 rows, 214 columns, 461 nonzeros
Variable types: 38 continuous, 176 integer (31 binary)

Root relaxation: objective 1.357563e+07, 65 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 1.3576e+07    0    8 1.6559e+07 1.3576e+07  18.0%     -    0s
H    0     0                    1.358140e+07 1.3576e

In [31]:
for v in m.getVars():
        print('%s %g' % (v.varName, v.x))
print('Obj: %g' % m.objVal)

x_ijk[0,0,0] 55
x_ijk[0,0,1] 172
x_ijk[0,0,2] 194
x_ijk[0,1,0] 0
x_ijk[0,1,1] 0
x_ijk[0,1,2] 94
x_ijk[0,2,0] 0
x_ijk[0,2,1] 0
x_ijk[0,2,2] 185
x_ijk[0,3,0] 0
x_ijk[0,3,1] 0
x_ijk[0,3,2] -0
x_ijk[0,4,0] 0
x_ijk[0,4,1] -0
x_ijk[0,4,2] -0
x_ijk[0,5,0] -0
x_ijk[0,5,1] -0
x_ijk[0,5,2] -0
x_ijk[1,0,0] 53
x_ijk[1,0,1] 68
x_ijk[1,0,2] 185
x_ijk[1,1,0] 0
x_ijk[1,1,1] 0
x_ijk[1,1,2] 0
x_ijk[1,2,0] 0
x_ijk[1,2,1] 0
x_ijk[1,2,2] 0
x_ijk[1,3,0] 0
x_ijk[1,3,1] 0
x_ijk[1,3,2] -0
x_ijk[1,4,0] 0
x_ijk[1,4,1] -0
x_ijk[1,4,2] -0
x_ijk[1,5,0] -0
x_ijk[1,5,1] -0
x_ijk[1,5,2] -0
x_ijk[2,0,0] 179
x_ijk[2,0,1] 1
x_ijk[2,0,2] 49
x_ijk[2,1,0] 0
x_ijk[2,1,1] 0
x_ijk[2,1,2] 199
x_ijk[2,2,0] 0
x_ijk[2,2,1] 0
x_ijk[2,2,2] 200
x_ijk[2,3,0] 0
x_ijk[2,3,1] 0
x_ijk[2,3,2] -0
x_ijk[2,4,0] 0
x_ijk[2,4,1] -0
x_ijk[2,4,2] -0
x_ijk[2,5,0] -0
x_ijk[2,5,1] -0
x_ijk[2,5,2] -0
x_ijk[3,0,0] -0
x_ijk[3,0,1] 0
x_ijk[3,0,2] 0
x_ijk[3,1,0] 0
x_ijk[3,1,1] 0
x_ijk[3,1,2] 0
x_ijk[3,2,0] 0
x_ijk[3,2,1] 0
x_ijk[3,2,2] 0
x_ijk[3,3,0] 0
x_

In [21]:
for i in range(N):
    for j in range(M):
         print(f'InventoryExpr_ij[{i}][{j}]: {InventoryExpr_ij[i][j].getValue()}') 

InventoryExpr_ij[0][0]: 662.0
InventoryExpr_ij[0][1]: 607.0
InventoryExpr_ij[0][2]: 435.0
InventoryExpr_ij[0][3]: 241.0
InventoryExpr_ij[0][4]: 147.0
InventoryExpr_ij[0][5]: 0.0
InventoryExpr_ij[1][0]: 410.0
InventoryExpr_ij[1][1]: 357.0
InventoryExpr_ij[1][2]: 289.0
InventoryExpr_ij[1][3]: 104.0
InventoryExpr_ij[1][4]: 91.0
InventoryExpr_ij[1][5]: 0.0
InventoryExpr_ij[2][0]: 346.0
InventoryExpr_ij[2][1]: 167.0
InventoryExpr_ij[2][2]: 166.0
InventoryExpr_ij[2][3]: 117.0
InventoryExpr_ij[2][4]: 0.0
InventoryExpr_ij[2][5]: 0.0
InventoryExpr_ij[3][0]: 208.0
InventoryExpr_ij[3][1]: 258.0
InventoryExpr_ij[3][2]: 180.0
InventoryExpr_ij[3][3]: 49.0
InventoryExpr_ij[3][4]: 0.0
InventoryExpr_ij[3][5]: 0.0
InventoryExpr_ij[4][0]: 365.0
InventoryExpr_ij[4][1]: 303.0
InventoryExpr_ij[4][2]: 220.0
InventoryExpr_ij[4][3]: 130.0
InventoryExpr_ij[4][4]: 0.0
InventoryExpr_ij[4][5]: 0.0
InventoryExpr_ij[5][0]: 433.0
InventoryExpr_ij[5][1]: 356.0
InventoryExpr_ij[5][2]: 272.0
InventoryExpr_ij[5][3]: 145.

In [22]:
print(f'purchaseCost: {purchaseCostExpr.getValue()}')
print(f'FixedCost: {FixedCostExpr.getValue()}')
print(f'VariableCost: {VariableCostExpr.getValue()}')
print(f'InventoryCost: {InventoryCostExpr.getValue()}')
print(f'allContainerCost: {allContainerCostExpr.getValue()}')
print(f'BackOrderCost: {BackOrderCostExpr.getValue()}')
print(f'LostSaleCost: {LostSaleCostExpr.getValue()}')
print(f'Obj: { m.objVal}')

purchaseCost: 0.0
FixedCost: 0.0
VariableCost: 0.0
InventoryCost: 1561040.0
allContainerCost: 0.0
BackOrderCost: 0.0
LostSaleCost: 0.0
Obj: 1561040.0
